In [1]:
import napari
import scipy as sp
from roifile import ROI_TYPE, ROI_OPTIONS, roiwrite
from roifile import ImagejRoi
import numpy as np
import tifffile
import scipy.ndimage as ndi
from skimage.feature import peak_local_max
import os
import glob

In [2]:
viewer = napari.Viewer()

In [3]:
scale = [0.09, 0.031, 0.031]

In [4]:
def find_peaks(img, threshold=0.1, display=False):
    LoG = -ndi.gaussian_laplace(img/1000, sigma=[2,8,8])
    max_peaks = peak_local_max(LoG, indices=True, min_distance=3, threshold_rel=threshold)
    
    if display:
        viewer.add_image(LoG, blending='additive', colormap='magenta', scale=scale)
        viewer.add_points(max_peaks, n_dimensional=True, size=6, scale=scale)
    return max_peaks

def pair_finder(peaks, max_dist=6, scale=[0.12, 0.04, 0.04]):
    scaled_peaks = peaks.copy().astype(float)
    scaled_peaks[:,0] = scaled_peaks[:,0] * scale[0]
    scaled_peaks[:,1] = scaled_peaks[:,1] * scale[1]
    scaled_peaks[:,2] = scaled_peaks[:,2] * scale[2]
    
    d_matrix = sp.spatial.distance.squareform(sp.spatial.distance.pdist(scaled_peaks))
    d_matrix[d_matrix==0] = 10000
    
    min_pos = np.argmin(d_matrix, axis=0)
    distances = np.min(d_matrix, axis=0)
    sorte = np.argsort(distances)
    
    lst = []
    for a in np.arange(0,len(sorte)):
        if distances[sorte[a]]<max_dist:
            if any([all (peaks[sorte[a]]==b) for b in lst]) | any([all(peaks[min_pos][sorte[a]]==b) for b in lst]):
                lst
            elif np.abs(peaks[sorte[a]][0]-peaks[min_pos][sorte[a]][0])>3:
                lst
            else:
                lst.append(peaks[sorte[a]])
                lst.append(peaks[min_pos][sorte[a]])
    return lst

def find_peaks_in_file(fname, display=False, threshold=0.16):
    img = tifffile.imread(fname)
    peaks = find_peaks(img[:,1,:,:], threshold=threshold, display=display)
    filtered_peaks = pair_finder(peaks, max_dist=1.2, scale=scale)
    
    rois = []
    for pt in filtered_peaks:
        roi = ImagejRoi.frompoints([[pt[2],pt[1]]], z=(pt[0]), name='X'+str(pt[1]))
        roi.roitype = ROI_TYPE.POINT
        roi.options = ROI_OPTIONS.SHOW_LABELS
        rois.append(roi)
    zip_file = fname[0:-4]+'.zip'
    if os.path.exists(zip_file):
        os.remove(zip_file)
    roiwrite(zip_file, rois)
    if display:
        viewer.add_image(img, channel_axis=1, scale=scale)
        viewer.add_points(filtered_peaks, n_dimensional=True, size=6, scale=scale)

In [15]:
fnames = glob.glob('10032022/*SIM.tif')

In [16]:
fnames

['10032022\\Image 10_SIM.tif',
 '10032022\\Image 11_SIM.tif',
 '10032022\\Image 12_SIM.tif',
 '10032022\\Image 13_SIM.tif',
 '10032022\\Image 2_SIM.tif',
 '10032022\\Image 3_SIM.tif',
 '10032022\\Image 4_SIM.tif',
 '10032022\\Image 5_SIM.tif',
 '10032022\\Image 6_SIM.tif',
 '10032022\\Image 7_SIM.tif',
 '10032022\\Image 8_SIM.tif',
 '10032022\\Image 9_SIM.tif']

In [17]:
img = tifffile.imread(fnames[3])

In [19]:
fname = fnames[3]
print(fname)
viewer.layers.clear()
find_peaks_in_file(fname, display=True, threshold=0.25)

10032022\Image 13_SIM.tif


C:\Users\smc\AppData\Local\Temp\ipykernel_11136\3222335928.py:3: FutureWarning: indices argument is deprecated and will be removed in version 0.20. To avoid this warning, please do not use the indices argument. Please see peak_local_max documentation for more details.
  max_peaks = peak_local_max(LoG, indices=True, min_distance=3, threshold_rel=threshold)
C:\Users\smc\Anaconda3\envs\napari\lib\site-packages\numpy\core\numeric.py:2446: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())
C:\Users\smc\Anaconda3\envs\napari\lib\site-packages\numpy\core\numeric.py:2446: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())


In [20]:
for f in fnames:
    find_peaks_in_file(f, False, threshold=0.25)

C:\Users\smc\AppData\Local\Temp\ipykernel_11136\3222335928.py:3: FutureWarning: indices argument is deprecated and will be removed in version 0.20. To avoid this warning, please do not use the indices argument. Please see peak_local_max documentation for more details.
  max_peaks = peak_local_max(LoG, indices=True, min_distance=3, threshold_rel=threshold)
C:\Users\smc\Anaconda3\envs\napari\lib\zipfile.py:1517: UserWarning: Duplicate name: 'X391.roi'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
C:\Users\smc\Anaconda3\envs\napari\lib\zipfile.py:1517: UserWarning: Duplicate name: 'X814.roi'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
C:\Users\smc\Anaconda3\envs\napari\lib\zipfile.py:1517: UserWarning: Duplicate name: 'X107.roi'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
C:\Users\smc\Anaconda3\envs\napari\lib\zipfile.py:1517: UserWarning: Duplicate name: 'X959.roi'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
C:\Users\smc\A